In [ ]:
!pip install sentence_transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 255.8/255.8 kB 2.6 MB/s eta 0:00:00


In [ ]:
pip install fuzzywuzzy

In [ ]:
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from sentence_transformers import SentenceTransformer
from fuzzywuzzy import fuzz
from gensim.models import KeyedVectors

# Load pre-trained Sentence Transformer model
sentence_model = SentenceTransformer('all-MiniLM-L6-v2')

# Path to the GloVe file
glove_path = 'glove.6B.300d.txt'

# Load the GloVe model with Gensim
glove_model = KeyedVectors.load_word2vec_format(glove_path, binary=False, no_header=True)

# Function to calculate word similarity using GloVe
def glove_word_similarity(word, gt_words, threshold=0.5):
    # Check if the word is in the GloVe vocabulary
    if word in glove_model:
        relevance_scores = [glove_model.similarity(word, gt_word) for gt_word in gt_words if gt_word in glove_model]
        max_relevance = max(relevance_scores) if relevance_scores else 0
        return max_relevance >= threshold
    return False  # Word not in the model vocabulary

# Function to evaluate responses
def evaluate_responses(examples):
    results = []

    for example in examples:
        ground_truth = example['ground_truth']
        model_responses = example['model_responses']

        # Generate embeddings for the ground truth and transfer to CPU
        gt_embedding = sentence_model.encode(ground_truth, convert_to_tensor=True).cpu().numpy()
        gt_words = ground_truth.split()

        example_result = {'ground_truth': ground_truth, 'model_responses': {}}

        for model_name, response in model_responses.items():
            # Generate embeddings for the model response and transfer to CPU
            response_embedding = sentence_model.encode(response, convert_to_tensor=True).cpu().numpy()

            # Calculate cosine similarity using Sentence Transformer embeddings
            st_similarity_score = cosine_similarity(gt_embedding.reshape(1, -1), response_embedding.reshape(1, -1))[0][0]

            # Initialize lists for color coding
            color_coded_response = []
            extra_analysis = []

            # Evaluate each word in the response
            for word in response.split():
                # Check for exact match
                if word in gt_words:
                    color_coded_response.append(f"\033[92m{word}\033[0m")  # Green for matched words
                else:
                    # Fuzzy matching for relevance
                    max_fuzzy_score = max(fuzz.ratio(word, gt_word) for gt_word in gt_words)
                    if max_fuzzy_score >= 70:  # Threshold for fuzzy matching
                        color_coded_response.append(f"\033[93m{word}\033[0m")  # Yellow for fuzzy matches
                    # Use GloVe model for partial similarity check
                    elif glove_word_similarity(word, gt_words):
                        color_coded_response.append(f"\033[93m{word}\033[0m")  # Yellow for relevant words
                    else:
                        color_coded_response.append(f"\033[91m{word}\033[0m")  # Red for irrelevant words
                        extra_analysis.append(word)

            # Format the results
            example_result['model_responses'][model_name] = {
                'Color-coded Response': ' '.join(color_coded_response),
                'Sentence Transformer Similarity': st_similarity_score,
                'Extra Analysis': ', '.join(extra_analysis)
            }

        results.append(example_result)

    return results

# Example test cases
examples = [
    {
        'ground_truth': "Python is a versatile programming language used for web development, data science, and more.",
        'model_responses': {
            'Model 1': "Python is a popular programming language used for web development, data analysis, and machine learning.",
            'Model 2': "Python is a flexible language widely used for web development and data science.",
            'Model 3': "Python is a powerful language utilized in web development, data science, and many other fields.",
        }
    },
    {
        'ground_truth': "Artificial Intelligence is a branch of computer science that focuses on creating intelligent machines.",
        'model_responses': {
            'Model 1': "AI is a field of computer science focused on creating intelligent machines and algorithms.",
            'Model 2': "Artificial Intelligence deals with making machines capable of mimicking human intelligence.",
            'Model 3': "AI is the simulation of human intelligence by machines, especially computers.",
        }
    },
     {
        'ground_truth': "Climate change refers to long-term shifts and alterations in temperature and weather patterns.",
        'model_responses': {
            'Model 1': "Climate change causes long-term variations in weather patterns, affecting ecosystems and human life.",
            'Model 2': "Changes in climate patterns over extended periods are referred to as climate change.",
            'Model 3': "Climate change leads to shifts in temperature and long-term weather patterns.",
        }
    },
    {
        'ground_truth': "The stock market allows companies to raise money by selling shares to the public.",
        'model_responses': {
            'Model 1': "The stock market helps companies raise capital by selling ownership stakes to the public.",
            'Model 2': "The stock market is a place where shares of publicly listed companies are traded.",
            'Model 3': "The stock market enables businesses to generate capital by selling ownership shares to the public.",
        }
    },
    {
        'ground_truth': "Photosynthesis is the process by which plants use sunlight to synthesize foods from carbon dioxide and water.",
        'model_responses': {
            'Model 1': "Photosynthesis involves plants absorbing sunlight to produce food from carbon dioxide and water.",
            'Model 2': "Plants use sunlight during photosynthesis to convert carbon dioxide and water into food.",
            'Model 3': "In photosynthesis, plants capture light energy to transform carbon dioxide and water into food.",
        }
    },
    {
        'ground_truth': "Quantum computing harnesses quantum mechanics to perform complex computations more efficiently than classical computers.",
        'model_responses': {
            'Model 1': "Quantum computing leverages quantum mechanics to solve problems faster than traditional computers.",
            'Model 2': "Quantum computing utilizes quantum phenomena to perform computations more efficiently.",
            'Model 3': "Quantum computers solve complex tasks more quickly than classical computers through quantum mechanics.",
        }
    },
    {
        'ground_truth': "Blockchain is a decentralized ledger used to record transactions across multiple computers.",
        'model_responses': {
            'Model 1': "Blockchain is a distributed ledger that ensures secure and transparent transaction recording.",
            'Model 2': "Blockchain is a technology that records transactions in a decentralized and secure manner.",
            'Model 3': "Blockchain is a decentralized digital ledger that records transactions transparently across a network.",
        }
    },
    {
        'ground_truth': "Electric vehicles are cars that use electric motors instead of internal combustion engines.",
        'model_responses': {
            'Model 1': "Electric vehicles rely on electricity rather than fossil fuels for propulsion.",
            'Model 2': "Electric vehicles are powered by electric motors and reduce dependency on gasoline.",
            'Model 3': "Electric vehicles run on electricity and are seen as a cleaner alternative to traditional cars.",
        }
    },
    {
        'ground_truth': "5G technology is the next generation of wireless communication with higher speeds and lower latency.",
        'model_responses': {
            'Model 1': "5G offers faster communication speeds and lower latency than 4G technology.",
            'Model 2': "5G technology improves communication by offering higher speeds and lower latency than previous generations.",
            'Model 3': "5G delivers faster mobile internet with improved data speeds and low latency, ideal for real-time apps.",
        }
    },
    {
        'ground_truth': "Machine learning is a subset of AI that enables computers to learn from data without being explicitly programmed.",
        'model_responses': {
            'Model 1': "Machine learning enables AI systems to learn and improve performance based on data.",
            'Model 2': "Machine learning involves teaching machines to learn from data and make predictions or decisions.",
            'Model 3': "Machine learning allows computers to learn patterns from data and make decisions without being explicitly programmed.",
        }
    },
    {
        'ground_truth': "The internet of things (IoT) refers to a network of interconnected devices that can communicate with each other.",
        'model_responses': {
            'Model 1': "IoT connects devices to allow them to interact with each other and exchange information.",
            'Model 2': "The Internet of Things connects everyday devices to the internet for communication and data exchange.",
            'Model 3': "IoT involves smart devices connected through the internet, communicating and sharing data with each other.",
        }
    },
    {
        'ground_truth': "Cloud computing allows users to access computing resources and data storage over the internet.",
        'model_responses': {
            'Model 1': "Cloud computing provides scalable computing resources over the internet, accessible from anywhere.",
            'Model 2': "Cloud computing allows users to remotely access data storage and computational resources over the web.",
            'Model 3': "Cloud computing offers remote access to computing power and data storage via the internet, scaling as needed.",
        }
    },
    {
        'ground_truth': "Cybersecurity involves protecting computer systems and networks from theft, damage, or unauthorized access.",
        'model_responses': {
            'Model 1': "Cybersecurity focuses on defending digital systems from unauthorized access and data breaches.",
            'Model 2': "Cybersecurity aims to protect information systems from hackers and other threats.",
            'Model 3': "Cybersecurity protects networks and systems from cyberattacks and data theft, safeguarding sensitive information.",
        }
    },
    {
        'ground_truth': "Cryptocurrencies are digital or virtual currencies that use cryptography for security.",
        'model_responses': {
            'Model 1': "Cryptocurrencies like Bitcoin are decentralized and use cryptography to secure transactions.",
            'Model 2': "Cryptocurrencies operate using decentralized ledgers and provide secure, anonymous transactions.",
            'Model 3': "Cryptocurrencies are digital assets based on blockchain technology that offer secure and decentralized transactions.",
        }
    },
    {
        'ground_truth': "Augmented reality overlays digital information on the physical world to enhance the user experience.",
        'model_responses': {
            'Model 1': "Augmented reality adds digital elements to real-world environments to enhance user experiences.",
            'Model 2': "Augmented reality superimposes digital content on physical objects to enrich the user's perception.",
            'Model 3': "Augmented reality enhances the physical world by adding interactive digital elements to the user's surroundings.",
        }
    }

]

# Evaluate the test cases
results = evaluate_responses(examples)

# Print the results
for result in results:
    print("Ground Truth:", result['ground_truth'])
    for model_name, response in result['model_responses'].items():
        print(f"{model_name}:")
        print("Color-coded Response:", response['Color-coded Response'])
        print("Extra Analysis:", response['Extra Analysis'])
        print("Sentence Transformer Similarity:", response['Sentence Transformer Similarity'])
    print()

/usr/local/lib/python3.10/dist-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange
/usr/local/lib/python3.10/dist-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is reco

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Ground Truth: Python is a versatile programming language used for web development, data science, and more.
Model 1:
Color-coded Response: Python is a popular programming language used for web development, data analysis, and machine learning.
Extra Analysis: popular, analysis,, machine, learning.
Sentence Transformer Similarity: 0.9524534
Model 2:
Color-coded Response: Python is a flexible language widely used for web development and data science.
Extra Analysis: flexible, widely
Sentence Transformer Similarity: 0.94979376
Model 3:
Color-coded Response: Python is a powerful language utilized in web development, data science, and many other fields.
Extra Analysis: powerful, fields.
Sentence Transformer Similarity: 0.9573464

Ground Truth: Artificial Intelligence is a branch of computer science that focuses on creating intelligent machines.
Model 1:
Color-coded Response: AI is a field of computer science focused on creating intelligent machines and algorithms.
Extra Analysis: AI, field, a

In [ ]:
!wget http://nlp.stanford.edu/data/glove.6B.zip
!unzip glove.6B.zip

--2024-10-28 13:40:29--  http://nlp.stanford.edu/data/glove.6B.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://nlp.stanford.edu/data/glove.6B.zip [following]
--2024-10-28 13:40:29--  https://nlp.stanford.edu/data/glove.6B.zip
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://downloads.cs.stanford.edu/nlp/data/glove.6B.zip [following]
--2024-10-28 13:40:29--  https://downloads.cs.stanford.edu/nlp/data/glove.6B.zip
Resolving downloads.cs.stanford.edu (downloads.cs.stanford.edu)... 171.64.64.22
Connecting to downloads.cs.stanford.edu (downloads.cs.stanford.edu)|171.64.64.22|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 862182613 (822M) [application/zip]
Saving to: ‘glove.6B.zip’

glov